# Postgres crime reports database 

The goal of this project is to create a database named crimes_db with a table – boston_crimes – with 

appropriate datatypes for storing the data from the boston.csv file. 

You will be creating the table inside a schema named crimes. You will also create the readonly and 

readwrite groups with the appropriate privileges. 

Finally, you will also need to create one user for each of these groups.


I will start by creating a database for storing our crime data as well as a schema for containing the tables

In [19]:
import psycopg2
conn = psycopg2.connect("dbname=dq user=dq")
conn.autocommit = "TRUE" #for creating a database
cur = conn.cursor()
cur.execute("CREATE DATABASE crime_db")
conn.close() #close the connection to the dq database

ProgrammingError: database "crime_db" already exists


In [20]:
conn = psycopg2.connect("dbname=crime_db user=dq") #connect to the new database
conn.autocommit = 'TRUE'
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")

Reading the column names from the boston.csv file as well as the first row. This way, I will have them at hand throughout this project so I can easily refer to them.

In [4]:
import csv
with open("boston.csv", "r") as f:
    reader = csv.reader(f)
    col_headers = next(reader)
    first_row = next(reader)

I will create a function — get_col_set() — that, given the name of a CSV file and a column index (starting at 0), computes a Python set with all distinct values contained in that column.

This function will be useful for two reasons:

Checking whether an enumerated datatype might be a good choice for representing a column
Computing the maximum length of any text-like column to select appropriate sizes for VARCHAR columns

In [5]:
def get_col_set(csv_filename, col_index):
    import csv
    values = set()
    with open(csv_filename, "r") as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
        return values
    
for i in range(len(col_headers)):
    values = get_col_set("boston.csv",i)
    print(col_headers[i],len(values), sep='\t')

incident_number	298329
offense_code	219
description	239
date	1177
day_of_the_week	7
lat	18177
long	18177


In [6]:
print(col_headers)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']


In [7]:
len_word = get_col_set("boston.csv",2)
length = 0
for row in len_word:
    length = max(length, len(row))
print(length)

58


Finding the Maximum Length

Important aspect is to know the longest word in any column containing textual data.

Let's compute the maximum length of each value in the description column.

In [21]:
print(col_headers)
print(first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long']
['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


incident_number	298329

offense_code	219

description	239

date	1177

day_of_the_week	7

lat	18177

long	18177

incident_number and offense_code will be integers

description string

date date 

day_of_the_week enumerated - there are limited and specific string

lat and long decimal - due to precision

In [22]:
# create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );""")

Loading the Data

I will use the cursor.copy_expert() method as it is a fast way of uploading data into postgres database

In [23]:
#load the data from Boston.csv into crimes.boston_crimes 

with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)


In [24]:
#print number of rows to ensure data was uploaded
cur.execute("SELECT * FROM crimes.boston_crimes")
print(len(cur.fetchall()))

298329


Revoking Public Privileges

The goal is to create the two user groups: readonly and readwrite. By following the least privilege principle, the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

We also need to revoke all privileges in the newly created schema. Doing this also makes it so that we do not need to revoke the privileges when we create users and groups because unless specified otherwise, privileges are not granted by default.

In [13]:
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")

Creating User Groups

In [28]:
cur.execute("CREATE GROUP readonly NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly;")
cur.execute("GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly;")

In [29]:
cur.execute("CREATE GROUP readwrite NOLOGIN;")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite;")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite ;")
cur.execute("GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite;")

Creating Users

In [30]:
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1';")
cur.execute("GRANT readonly TO data_analyst")

In [34]:
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2';")
cur.execute("GRANT readwrite TO data_scientist")

In [ ]:
# close the old connection to test with a brand new connection
conn.close()

conn = psycopg2.connect(dbname="crime_db", user="dq")
cur = conn.cursor()
# check users and groups
cur.execute("""
    SELECT rolname, rolsuper, rolcreaterole, rolcreatedb, rolcanlogin FROM pg_roles
    WHERE rolname IN ('readonly', 'readwrite', 'data_analyst', 'data_scientist');
""")
for user in cur:
    print(user)
print()
# check privileges
cur.execute("""
    SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee IN ('readonly', 'readwrite');
""")
for user in cur:
    print(user)
conn.close()

In [ ]:
Testing

It's a good practice to test that everything is configured as expected when you finish setting up the database. We can query Postgres internal tables to see this information.